In [1]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
ppg_file =(os.listdir("./dataverse_files/ppg"))
labels_file =( os.listdir("./dataverse_files/labels"))
print(len(labels_file))
print(len(ppg_file))
x = np.array([ppg_file,labels_file])
pid = set()
for i in ppg_file:
    pid.add(i[:-8])

labels_pid = set()
for i in pid:
    path_l = f'{i}_labels.npy'
    # print(path_l)
    if path_l in labels_pid:
        print(path_l)


Xt = []
Yt = []
Xcv = []
for i in range(len(ppg_file)):
    ippg = np.load(f"dataverse_files/ppg/{ppg_file[i]}")
    ilabels = np.load(f"dataverse_files/labels/{ppg_file[i][:-8]}_labels.npy")
    # X[30*i:30*i+30,:] = ippg
    mean_dev = 0
    sum_dev = 0
    for j in ippg:
        sum_dev+=np.std(j)

    mean_dev = sum_dev/30
    Xc = []
    for j in range(30):
        if np.std(ippg[j])>=0.75*mean_dev:
            Xt.append(ippg[j])
            Yt.append(ilabels[j])
            Xc.append(ippg[j])
        
    Xcv.append(np.array(Xc).flatten())
    # Y[30*i:30*i+30,:] = ilabels
    

X = np.array(Xt)
Y = np.array(Yt)
# Xz = np.array(Xcv)

1524
1524


In [10]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_array(arr, mode='minmax', range_min=-1, range_max=1):
    mean = np.mean(arr)
    max = np.max(arr)
    min = np.min(arr)

    normalized_array = (arr-mean)/(max-min)
    return normalized_array



In [ ]:
import tensorflow as tf


In [21]:
from scipy.signal import stft
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

class STFT_LR :
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y
        self.X_normalized = np.zeros_like(X)
    def print_data(self):
        print(self.features)

    def normalize_data(self):
        for i in range(X.shape[0]):
            mean = np.mean(X[i])
            max = np.max(X[i])
            min = np.min(X[i])
            self.X_normalized[i] = (X[i] - mean)/(max-min)
    
    def normalize_sample(self,arr):
        min = np.min(arr)
        mean = np.mean(arr)
        max = np.max(arr)
        normie = (arr-mean)/(max-min)
        return normie

    def stft_coeff(self):
        features = []
        for signal in X:
            f, t, Zxx = stft(signal, nperseg=125)
            features.append(np.abs(Zxx).flatten()[:1000])
        self.features = np.array(features)

    def get_features(self,signal):
        features =[]
        f,t,Zxx = stft(signal,nperseg=125)
        features.append(np.abs(Zxx).flatten()[:1000])
        return features

    def split_data(self):
        self.X_train_split, self.X_test_split, self.y_train_split, self.y_test_split = train_test_split(self.features, self.Y, test_size=0.2, random_state=42)

    def train_model(self):
        self.normalize_data()
        self.stft_coeff()
        self.split_data()
        self.model = LinearRegression()
        self.model.fit(self.X_train_split, self.y_train_split)
        # Predictions
        y_pred = self.model.predict(self.X_test_split)

        # Evaluation
        mae = mean_absolute_error(self.y_test_split, y_pred)
        mse = mean_squared_error(self.y_test_split, y_pred)
        print(f"MAE: {mae}, MSE: {mse}")
        del self.X
        del self.Y
        del self.X_train_split
        del self.X_test_split
        del self.y_train_split
        del self.y_test_split
        del self.features
    
    def predict(self,data_sample):
        n_sample = self.normalize_sample(data_sample)
        features = self.get_features(n_sample)
        predictions = self.model.predict(features)
        return predictions
        
    

In [22]:
t = STFT_LR(X,Y)
t.train_model()

MAE: 11.38327316300784, MSE: 216.76268535617263


In [26]:
i = -1
pred = t.predict(X[i])
print(pred,Y[i])

[[123.45645219  54.31031672]] [76.2 48.9]


In [50]:
import pickle as pkl
with open("models/pipeline.pkl", 'wb') as f:
    pkl.dump(t,f)

In [24]:
import numpy as np
from scipy.signal import stft
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

class STFT_LR:
    def __init__(self, X=None, Y=None):
        self.X = X
        self.Y = Y
        self.model = None
        self.nperseg = 125
        self.n_features = 1000  # number of STFT coefficients to keep

    def normalize_sample(self, arr):
        min_val = np.min(arr)
        max_val = np.max(arr)
        mean_val = np.mean(arr)
        return (arr - mean_val) / (max_val - min_val + 1e-6)

    def get_features(self, signal):
        signal = self.normalize_sample(signal)
        _, _, Zxx = stft(signal, nperseg=self.nperseg)
        features = np.abs(Zxx).flatten()[:self.n_features]
        return features

    def prepare_features(self):
        self.features = np.array([self.get_features(sig) for sig in self.X])

    def split_data(self):
        self.X_train_split, self.X_test_split, self.y_train_split, self.y_test_split = train_test_split(
            self.features, self.Y, test_size=0.2, random_state=42
        )

    def train_model(self):
        self.prepare_features()
        self.split_data()
        self.model = LinearRegression()
        self.model.fit(self.X_train_split, self.y_train_split)
        y_pred = self.model.predict(self.X_test_split)
        mae = mean_absolute_error(self.y_test_split, y_pred)
        mse = mean_squared_error(self.y_test_split, y_pred)
        print(f"MAE: {mae}, MSE: {mse}")

    def predict(self, signal):
        features = self.get_features(signal).reshape(1, -1)
        return self.model.predict(features)

    def save_model(self, path="stft_lr_model.pkl"):
        joblib.dump({
            "model": self.model,
            "nperseg": self.nperseg,
            "n_features": self.n_features
        }, path)
        print(f"Model saved to {path}")

    @staticmethod
    def load_model(path="stft_lr_model.pkl"):
        data = joblib.load(path)
        model = STFT_LR()
        model.model = data["model"]
        model.nperseg = data["nperseg"]
        model.n_features = data["n_features"]
        return model


In [25]:
# Training
model = STFT_LR(X, Y)
model.train_model()
model.save_model("stft_lr_model.pkl")

# Prediction later
loaded_model = STFT_LR.load_model("stft_lr_model.pkl")
pred = loaded_model.predict(X[-1])
print("Predicted value:", pred)


MAE: 11.212909357551862, MSE: 209.31230631116486
Model saved to stft_lr_model.pkl
Predicted value: [[111.7678073   64.65520955]]


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

from keras.saving import register_keras_serializable


@register_keras_serializable()
class STFTLayer(tf.keras.layers.Layer):
    def __init__(self, nperseg, frame_step, fft_length, n_features, **kwargs):
        super().__init__(**kwargs)
        self.nperseg = nperseg
        self.frame_step = frame_step
        self.fft_length = fft_length
        self.n_features = n_features

    def call(self, x):
        x = tf.cast(x, tf.float32)
        mean = tf.reduce_mean(x, axis=1, keepdims=True)
        max_ = tf.reduce_max(x, axis=1, keepdims=True)
        min_ = tf.reduce_min(x, axis=1, keepdims=True)
        normed = (x - mean) / (max_ - min_ + 1e-6)

        stft_out = tf.signal.stft(normed,
                                  frame_length=self.nperseg,
                                  frame_step=self.frame_step,
                                  fft_length=self.fft_length,
                                  pad_end=True)
        mag = tf.abs(stft_out)
        flat = tf.reshape(mag, [tf.shape(mag)[0], -1])
        return flat[:, :self.n_features]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.n_features)



class STFTLinearTF:
    def __init__(self, nperseg=125, frame_step=25, fft_length=125, n_features=3900, signal_length=3750):
        self.nperseg = nperseg
        self.frame_step = frame_step
        self.fft_length = fft_length
        self.n_features = n_features
        self.signal_length = signal_length
        self.model = self._build_model()

    def _stft_layer(self, x):
        x = tf.cast(x, tf.float32)
        mean = tf.reduce_mean(x, axis=1, keepdims=True)
        max_ = tf.reduce_max(x, axis=1, keepdims=True)
        min_ = tf.reduce_min(x, axis=1, keepdims=True)
        normed = (x - mean) / (max_ - min_ + 1e-6)

        stft_out = tf.signal.stft(normed,
                                  frame_length=self.nperseg,
                                  frame_step=self.frame_step,
                                  fft_length=self.fft_length,
                                  pad_end=True)
        mag = tf.abs(stft_out)
        flat = tf.reshape(mag, [tf.shape(mag)[0], -1])
        return flat[:, :self.n_features]

    def _build_model(self):
        inputs = tf.keras.Input(shape=(self.signal_length,))
        features = STFTLayer(self.nperseg, self.frame_step, self.fft_length, self.n_features)(inputs)
        output = tf.keras.layers.Dense(64,use_bias=True)(features)
        output = tf.keras.layers.Dense(32,use_bias=True)(output)
        output = tf.keras.layers.Dense(1,use_bias=True)(output)
        model = tf.keras.Model(inputs, output)
        model.compile(optimizer='adam', loss='mse')
        return model

    def train(self, X, y, epochs=20, batch_size=3):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
        for i in range(5):
            self.model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=batch_size, verbose=1)
        print("Training completed.")

    def predict(self, signal):
        signal = np.array(signal).reshape(1, self.signal_length).astype(np.float32)
        return self.model.predict(signal)

    def save_model(self, path="stft_tf_model.keras"):
        self.model.save(path)
        print(f"Model saved at {path}")

    @staticmethod
    def load_model(path="stft_tf_model.keras"):
        instance = STFTLinearTF()
        instance.model = tf.keras.models.load_model(path)
        print(f"Model loaded from {path}")
        return instance


In [61]:
# X = np.random.randn(100, 3750).astype(np.float32)
y = Y.T[0].reshape(-1,1)

stft_model = STFTLinearTF()
stft_model.train(X, y)

stft_model.save_model()


UnboundLocalError: cannot access local variable 'output' where it is not associated with a value

In [53]:

# Load and predict
loaded = STFTLinearTF.load_model()
i = -32
print("Prediction:", loaded.predict(X[i]), f"Original :{Y[i]} " )


Model loaded from stft_tf_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Prediction: [[107.46011]] Original :[131.   51.2] 


In [82]:
# X = np.random.randn(100, 3750).astype(np.float32)
y = np.random.uniform(50, 150, size=(100, 1)).astype(np.float32)


In [14]:
stft_model.predict(X[-1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


array([[103.4214]], dtype=float32)

In [ ]:
def _stft_layer( x):
    x = tf.cast(x, tf.float32)

    # Normalize input
    mean = tf.reduce_mean(x, axis=1, keepdims=True)
    max_ = tf.reduce_max(x, axis=1, keepdims=True)
    min_ = tf.reduce_min(x, axis=1, keepdims=True)
    normed = (x - mean) / (max_ - min_ + 1e-6)

    # Compute STFT
    stft = tf.signal.stft(normed,
                            frame_length=125,
                            frame_step=125,
                            fft_length=125,
                            pad_end=True)

    magnitude = tf.abs(stft)  # Take magnitude of STFT
    flat = tf.reshape(magnitude, [-1, 1000])  # Flatten STFT output
    return flat


In [32]:
tt = _stft_layer(X[:2])

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 3780 values, but the requested shape has 1000 [Op:Reshape] name: 

In [27]:
tt.shape

(125, 1)

In [28]:
X.shape

(40450, 3750)